In [12]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap


# Load and preprocess your data (same as before)
csv_path = "/Users/jamesjackson/Documents/liverpool_crime_analysis/csv_files/clean_data_final.csv"
df = pd.read_csv(csv_path, parse_dates=['month'])
df['month_str'] = df['month'].dt.strftime('%b %Y')

# Crime groups mapping
crime_group_map = {
    'Violence and sexual offences': 'Violent / Sexual',
    'Criminal damage and arson': 'Violent / Sexual',
    'Vehicle crime': 'Vehicle Crime',
    'Burglary': 'Theft / Burglary / Robbery',
    'Theft from the person': 'Theft / Burglary / Robbery',
    'Shoplifting': 'Theft / Burglary / Robbery',
    'Other theft': 'Theft / Burglary / Robbery',
    'Bicycle theft': 'Theft / Burglary / Robbery',
    'Drugs': 'Drug-Related Crime',
    'Anti-social behaviour': 'Anti-Social Behaviour',
    'Other crime': 'Other Crime',
    'Possession of weapons': 'Weapons Offences'
}
# Map crime_type to groups; unmapped ones become 'Unknown'
df['crime_group'] = df['crime_type'].map(crime_group_map).fillna('Unknown')

# Location grouping
top_locations = [
    "On or near Parking Area",
    "On or near Supermarket",
    "On or near Shopping Area",
    "On or near Petrol Station",
    "On or near Nightclub",
    "On or near Sports/Recreation Area",
    "On or near Further/Higher Educational Building",
    "On or near Hospital",
    "On or near Police Station"
]
df['location_group'] = df['location'].apply(lambda x: x if x in top_locations else 'Other').fillna('Unknown')

# Region grouping
def extract_region(lsoa_name):
    if isinstance(lsoa_name, str):
        return lsoa_name.split()[0]
    return 'Unknown'
df['region'] = df['lsoa_name'].apply(extract_region)
top_regions = df['region'].value_counts().nlargest(5).index.tolist()
df['region_group'] = df['region'].apply(lambda x: x if x in top_regions else 'Other').fillna('Unknown')

# Outcome groups
positive_outcomes = [
    "Action to be taken by another organisation",
    "Offender given a drugs possession warning",
    "Suspect charged as part of another case",
    "Offender given a caution",
    "Local resolution"
]
neutral_outcomes = [
    "Status update unavailable",
    "Under investigation",
    "Awaiting court outcome",
    "Court result unavailable"
]
negative_outcomes = [
    "Unable to prosecute suspect",
    "Investigation complete; no suspect identified",
    "Further action is not in the public interest",
    "Further investigation is not in the public interest",
    "Formal action is not in the public interest"
]
def map_outcome_group(outcome):
    if outcome in positive_outcomes:
        return 'Positive'
    elif outcome in neutral_outcomes:
        return 'Neutral'
    elif outcome in negative_outcomes:
        return 'Negative'
    elif pd.isna(outcome):
        return 'Unknown'
    else:
        return 'Unknown'
df['outcome_group'] = df['last_outcome_category'].apply(map_outcome_group)

# Options for dropdowns with 'All'
def options_with_all(series):
    return ['All'] + sorted(series.dropna().unique().tolist())

crime_group_options = options_with_all(df['crime_group'])
outcome_group_options = options_with_all(df['outcome_group'])
month_options = options_with_all(df['month_str'])
region_options = options_with_all(df['region_group'])
location_options = options_with_all(df['location_group'])

# --- Define Widgets for EACH filter group ---

# 1. KPI Filters
kpi_filters = {
    'crime_group': widgets.Dropdown(options=crime_group_options, description='Crime Group:'),
    'outcome_group': widgets.Dropdown(options=outcome_group_options, description='Outcome Group:'),
    'month': widgets.Dropdown(options=month_options, description='Month:'),
    'region': widgets.Dropdown(options=region_options, description='Region:'),
    'location': widgets.Dropdown(options=location_options, description='Location:')
}

# 2. Outcome Group Pie Filters
outcome_pie_filters = {
    'crime_group': widgets.Dropdown(options=crime_group_options, description='Crime Group:'),
    'month': widgets.Dropdown(options=month_options, description='Month:'),
    'location': widgets.Dropdown(options=location_options, description='Location:'),
    'region': widgets.Dropdown(options=region_options, description='Region:')
}

# 3. Crime Group Pie Filters
crime_pie_filters = {
    'outcome_group': widgets.Dropdown(options=outcome_group_options, description='Outcome Group:'),
    'region': widgets.Dropdown(options=region_options, description='Region:'),
    'location': widgets.Dropdown(options=location_options, description='Location:'),
    'month': widgets.Dropdown(options=month_options, description='Month:')
}

# 4. Crime Type Distibution Filters

crime_type_filters = {
    'region': widgets.Dropdown(options=region_options, description='Region:'),
    'location': widgets.Dropdown(options=location_options, description='Location:'),
    'month': widgets.Dropdown(options=month_options, description='Month:'),
    'outcome': widgets.Dropdown(options=outcome_group_options, description='Outcome:')
}

# 5. Regional Crime Volume Filters

regional_crime_filters = {
    'month': widgets.Dropdown(options=month_options, description='Month:'),
    'outcome': widgets.Dropdown(options=outcome_group_options, description='Outcome:'),
    'location': widgets.Dropdown(options=location_options, description='Location:'),
    'crime_group': widgets.Dropdown(options=crime_group_options, description='Crime Group:')
}

# 6. Crime Over Time Filters

crime_over_time_filters = {
    'crime_group': widgets.Dropdown(options=crime_group_options, description='Crime Group:'),
    'region': widgets.Dropdown(options=region_options, description='Region:'),
    'location': widgets.Dropdown(options=location_options, description='Location:'),
    'outcome': widgets.Dropdown(options=outcome_group_options, description='Outcome:')
}

# 7. Positive Outcome Rates Filters

positive_outcomes_by_region_filters = {
    'month': widgets.Dropdown(options=month_options, description='Month:'),
    'location': widgets.Dropdown(options=location_options, description='Location:'),
    'crime_group': widgets.Dropdown(options=crime_group_options, description='Crime Group:')
}
    
# 8. Crime Group by Location Filters

crime_group_by_location_filters = {
    'month': widgets.Dropdown(options=month_options, description='Month:'),
    'region': widgets.Dropdown(options=region_options, description='Region:'),
    'outcome': widgets.Dropdown(options=outcome_group_options, description='Outcome:')
}

# 9. Merseyside Crime Hexbin Filters

crime_hexbin_filters = {
    'crime_group': widgets.Dropdown(options=crime_group_options, description='Crime Group:'),
    'outcome_group': widgets.Dropdown(options=outcome_group_options, description='Outcome Group:'),
    'month': widgets.Dropdown(options=month_options, description='Month:'),
    'region': widgets.Dropdown(options=region_options, description='Region:'),
    'location': widgets.Dropdown(options=location_options, description='Location:')
}

# --- Plot functions ---

def plot_kpi():
    filtered = df.copy()
    if kpi_filters['crime_group'].value != 'All':
        filtered = filtered[filtered['crime_group'] == kpi_filters['crime_group'].value]
    if kpi_filters['outcome_group'].value != 'All':
        filtered = filtered[filtered['outcome_group'] == kpi_filters['outcome_group'].value]
    if kpi_filters['month'].value != 'All':
        filtered = filtered[filtered['month_str'] == kpi_filters['month'].value]
    if kpi_filters['region'].value != 'All':
        filtered = filtered[filtered['region_group'] == kpi_filters['region'].value]
    if kpi_filters['location'].value != 'All':
        filtered = filtered[filtered['location_group'] == kpi_filters['location'].value]

    total = len(filtered)
    fig = go.Figure(go.Indicator(
        mode="number",
        value=total,
        title={"text": "Total Crimes Recorded"},
        number={'valueformat': ','}
    ))
    fig.update_layout(height=200,width=1100)
    
    return fig

def plot_outcome_pie():
    filtered = df.copy()
    if outcome_pie_filters['crime_group'].value != 'All':
        filtered = filtered[filtered['crime_group'] == outcome_pie_filters['crime_group'].value]
    if outcome_pie_filters['month'].value != 'All':
        filtered = filtered[filtered['month_str'] == outcome_pie_filters['month'].value]
    if outcome_pie_filters['location'].value != 'All':
        filtered = filtered[filtered['location_group'] == outcome_pie_filters['location'].value]
    if outcome_pie_filters['region'].value != 'All':
        filtered = filtered[filtered['region_group'] == outcome_pie_filters['region'].value]

    counts = filtered['outcome_group'].value_counts(normalize=True) * 100
    labels = counts.index.tolist()
    values = counts.values.tolist()

    colors_map = {
        'Positive': '#A8D5BA',
        'Neutral': '#F7E8A3',
        'Negative': '#F7B7A3',
        'Unknown': '#CCCCCC'
    }
    colors = [colors_map.get(label, '#CCCCCC') for label in labels]

    fig = go.Figure(go.Pie(
        labels=labels,
        values=values,
        hole=0.4,
        marker=dict(colors=colors, line=dict(color='black', width=1.5)),
        textinfo='percent',
        hoverinfo='label+percent'
    ))
    fig.update_layout(title_text='Outcome Group Distribution', width=1100, height=700)
    return fig

def plot_crime_group_pie():
    filtered = df.copy()
    if crime_pie_filters['outcome_group'].value != 'All':
        filtered = filtered[filtered['outcome_group'] == crime_pie_filters['outcome_group'].value]
    if crime_pie_filters['region'].value != 'All':
        filtered = filtered[filtered['region_group'] == crime_pie_filters['region'].value]
    if crime_pie_filters['location'].value != 'All':
        filtered = filtered[filtered['location_group'] == crime_pie_filters['location'].value]
    if crime_pie_filters['month'].value != 'All':
        filtered = filtered[filtered['month_str'] == crime_pie_filters['month'].value]

    counts = filtered['crime_group'].value_counts()
    if counts.empty:
        return None

    labels = counts.index.tolist()
    values = counts.values.tolist()

    # You can define colors for each group to be consistent
    colors_map = {
    'Violent / Sexual': '#F7B7A3',
    'Theft / Burglary / Robbery': '#A8D5BA',
    'Vehicle Crime': '#D5A3F7',
    'Drug-Related Crime': '#E2F0CB',
    'Anti-Social Behaviour': '#F7E8A3',
    'Other Crime': '#F1CBFF',
    'Weapons Offences': '#F7C5A3',
    'Unknown': '#C0C0C0',
}
    colors = [colors_map.get(label, '#CCCCCC') for label in labels]

    fig = go.Figure(go.Pie(
        labels=labels,
        values=values,
        hole=0.4,
        textinfo='percent',
        hoverinfo='label+percent',
        marker=dict(colors=colors, line=dict(color='black', width=1.5))
    ))
    fig.update_layout(title_text="Crime Group Distribution", width=1100, height=700,
                      legend=dict(orientation="h", y=-0.2, x=0.5, xanchor='center'))
    return fig

def plot_crime_type():
    dff = df.copy()

    region = crime_type_filters['region'].value
    location = crime_type_filters['location'].value
    month = crime_type_filters['month'].value
    outcome = crime_type_filters['outcome'].value

    if region != 'All':
        dff = dff[dff['region_group'] == region]
    if location != 'All':
        dff = dff[dff['location_group'] == location]
    if month != 'All':    
        dff = dff[dff['month_str'] == month]
    if outcome != 'All':
        dff = dff[dff['outcome_group'] == outcome]

    crime_type_counts = dff['crime_type'].value_counts()

    with crime_type_output:
        crime_type_output.clear_output()

        if crime_type_counts.empty:
            print("No data for selected filters.")
            return

        pastel_colors = [
            '#D5A3F7',   # purple-ish
            '#E2F0CB',   # light green
            '#F7E8A3',   # yellow
            '#F1CBFF',   # light purple
            '#F7C5A3',   # peach
            '#C7CEEA',   # blue-ish
            '#B5EAD7',   # mint
            '#A3B4F7',   # blue
            '#C0C0C0',   # grey
            '#FFDAC1',   # pale peach
        ]

        # Mapping crime_type to crime_group from original df
        crime_type_to_group = df.set_index('crime_type')['crime_group'].to_dict()

        used_colors = set()
        def get_color(crime_type):
            group = crime_type_to_group.get(crime_type, None)
            if group == 'Violent / Sexual':
                return '#F7B7A3'  # coral red
            elif group == 'Theft / Burglary / Robbery':
                return '#A8D5BA'  # pastel green
            else:
                for c in pastel_colors:
                    if c not in used_colors:
                        used_colors.add(c)
                        return c
                return '#CCCCCC'  # fallback grey if colors run out

        colors = [get_color(ct) for ct in crime_type_counts.index]

        fig = go.Figure(data=go.Bar(
            x=crime_type_counts.index,
            y=crime_type_counts.values,
            marker=dict(
                color=colors,
                line=dict(color='black', width=1.5)
            )
        ))

        fig.update_layout(
            title="Distribution of Crime Types",
            xaxis_title="Crime Type",
            yaxis_title="Number of Crimes",
            xaxis_tickangle=-45,
            margin=dict(l=50, r=50, t=50, b=150),
            width=1100,
            height=700,
        )

        fig.show()

def plot_regional_crime():
    with regional_crime_output:
        regional_crime_output.clear_output()

        dff = df.copy()

        # Read current filter values
        month_filter = regional_crime_filters['month'].value
        outcome_filter = regional_crime_filters['outcome'].value
        location_filter = regional_crime_filters['location'].value
        crime_group_filter = regional_crime_filters['crime_group'].value

        # Apply filters
        if month_filter != 'All':
            dff = dff[dff['month_str'] == month_filter]
        if outcome_filter != 'All':
            dff = dff[dff['outcome_group'] == outcome_filter]
        if location_filter != 'All':
            dff = dff[dff['location_group'] == location_filter]
        if crime_group_filter != 'All':
            dff = dff[dff['crime_group'] == crime_group_filter]

        dff = dff[dff['region_group'] != 'Other']
        region_counts = dff['region_group'].value_counts().sort_values(ascending=False)

        fig = go.Figure()

        if region_counts.empty:
            fig.update_layout(title="No data for selected filters")
        else:
            pastel_colors = ['#F7B7A3', '#A8D5BA', '#A3B4F7', '#F7E8A3', '#D5A3F7']
            
            x_labels = [label if label != 'St.' else 'St Helens' for label in region_counts.index]
            
            fig = go.Figure(data=go.Bar(
                x=x_labels,
                y=region_counts.values,
                marker=dict(
                    color=pastel_colors[:len(region_counts)],
                    line=dict(color='#333333', width=1.5)
                )
            ))

            fig.update_layout(
                width=1100,
                height=700,
                title="Crime Volume by Region",
                xaxis_title="Region",
                yaxis_title="Number of Crimes",
                margin=dict(l=50, r=50, t=50, b=50),
            )

        fig.show()

def plot_crime_over_time():
    with crime_over_time_output:
        crime_over_time_output.clear_output()

        dff = df.copy()

        # Read filter values
        crime_group = crime_over_time_filters['crime_group'].value
        region = crime_over_time_filters['region'].value
        location = crime_over_time_filters['location'].value
        outcome = crime_over_time_filters['outcome'].value

        # Apply filters
        if crime_group != 'All':
            dff = dff[dff['crime_group'] == crime_group]
        if region != 'All':
            dff = dff[dff['region_group'] == region]
        if location != 'All':
            dff = dff[dff['location_group'] == location]
        if outcome != 'All':
            dff = dff[dff['outcome_group'] == outcome]

        # Total crimes (unfiltered)
        total_counts = df.groupby('month').size().sort_index()

        # Filtered crimes
        filtered_counts = dff.groupby('month').size().reindex(total_counts.index, fill_value=0)
        months_str = total_counts.index.strftime('%b %Y')

        pastel_colors = [
            '#D5A3F7', '#E2F0CB', '#F7E8A3', '#F1CBFF',
            '#F7C5A3', '#F7B7A3', '#A8D5BA', '#C7CEEA',
            '#B5EAD7', '#A3B4F7', '#C0C0C0', '#FFDAC1',
        ]
        bar_colors = [pastel_colors[m.month - 1] for m in total_counts.index]

        fig = go.Figure()

        fig.add_trace(go.Bar(
            x=months_str,
            y=total_counts.values,
            marker_color=bar_colors,
            marker_line_color='black',
            marker_line_width=1.5,
            name='Total Crimes',
            yaxis='y1',
            opacity=0.7
        ))

        fig.add_trace(go.Scatter(
            x=months_str,
            y=filtered_counts.values,
            mode='lines+markers',
            line=dict(color='#1F77B4', width=3),
            name='Filtered Trend',
            yaxis='y2'
        ))

        fig.update_layout(
            width=1100,
            height=700,
            showlegend=False,
            title={
                'text': "Trends in Total and Filtered Crime Counts Over Time",
                'font': {'size': 16}
            },
            yaxis=dict(
                range=[0, max(total_counts.values) * 1.1],
                title='Total Crimes',
                showline=True,
                linecolor='black',
                zeroline=True,
                zerolinecolor='black'
            ),
            yaxis2=dict(
                range=[0, max(filtered_counts.values) * 1.1],
                overlaying='y',
                side='right',
                title='Filtered Crimes',
                showline=True,
                linecolor='black',
                zeroline=True,
                zerolinecolor='black'
            ),
            xaxis=dict(
                title='Month',
                showline=True,
                linecolor='black',
                zeroline=True,
                zerolinecolor='black'
            )
        )

        fig.show()

def plot_positive_outcomes_by_region():
    dff = df[df['region_group'].notna()].copy()

    month_filter = positive_outcomes_by_region_filters['month'].value
    crime_group_filter = positive_outcomes_by_region_filters['crime_group'].value
    location_filter = positive_outcomes_by_region_filters['location'].value

    if month_filter != 'All':
        dff = dff[dff['month_str'] == month_filter]
    if crime_group_filter != 'All':
        dff = dff[dff['crime_group'] == crime_group_filter]
    if location_filter != 'All':
        dff = dff[dff['location_group'] == location_filter]

    grouped = dff.groupby('region_group').agg(
        total_crimes=('outcome_group', 'size'),
        positive_outcomes=('outcome_group', lambda x: (x == 'Positive').sum())
    )
    grouped['positive_rate'] = (grouped['positive_outcomes'] / grouped['total_crimes']) * 100
    grouped = grouped.sort_values('positive_rate', ascending=False)

    min_val, max_val = grouped['positive_rate'].min(), grouped['positive_rate'].max()

    def hex_to_rgb(hex_color):
        return np.array(mcolors.to_rgb(hex_color))

    def rgb_to_hex(rgb):
        return mcolors.to_hex(rgb)

    def get_gradient_color(value, min_val, max_val, low_hex, mid_hex, high_hex):
        low_rgb = hex_to_rgb(low_hex)
        mid_rgb = hex_to_rgb(mid_hex)
        high_rgb = hex_to_rgb(high_hex)
        ratio = (value - min_val) / (max_val - min_val) if max_val > min_val else 0.5

        if ratio < 0.5:
            interp = low_rgb + (mid_rgb - low_rgb) * (ratio / 0.5)
        else:
            interp = mid_rgb + (high_rgb - mid_rgb) * ((ratio - 0.5) / 0.5)
        return rgb_to_hex(interp)

    bar_colors = [
        get_gradient_color(v, min_val, max_val, '#F7B7A3', '#FFFFFF', '#A8D5BA')
        for v in grouped['positive_rate']
    ]

    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=grouped['positive_rate'],
        y=grouped.index,
        orientation='h',
        text=grouped['positive_rate'].round(1).astype(str) + '%',
        textposition='auto',
        marker_color=bar_colors,
        marker_line_color='black',
        marker_line_width=1.5
    ))

    fig.update_layout(
        title={
            'text': 'Positive Outcome Rates by Region',
            'font': {'size': 18}
        },
        xaxis=dict(
            title='Positive Outcome Rate (%)',
            range=[0, grouped['positive_rate'].max() * 1.1]
        ),
        yaxis=dict(
            title='Region',
            autorange='reversed'
        ),
        margin=dict(l=100, r=20, t=60, b=50),
        width=1100,
        height=700,
        template='plotly_white'
    )

    with positive_outcomes_by_region_output:
        positive_outcomes_by_region_output.clear_output()
        display(fig)

def plot_crime_group_by_location():
    dff = df.copy()

    # Retrieve filter values
    selected_month = crime_group_by_location_filters['month'].value
    selected_region = crime_group_by_location_filters['region'].value
    selected_outcome = crime_group_by_location_filters['outcome'].value

    # Apply filters
    if selected_month != 'All':
        dff = dff[dff['month_str'] == selected_month]
    if selected_region != 'All':
        dff = dff[dff['region'] == selected_region]
    if selected_outcome != 'All':
        dff = dff[dff['outcome_group'] == selected_outcome]

    # Focus on top locations
    dff = dff[dff['location_group'] != 'Other']

    # Define crime groups
    crime_groups_to_include = [
        'Violent / Sexual',
        'Theft / Burglary / Robbery',
        'Anti-Social Behaviour',
        'Drug-Related Crime'
    ]
    dff = dff[dff['crime_group'].isin(crime_groups_to_include)]

    # Group and reshape data
    grouped = dff.groupby(['location_group', 'crime_group']).size().reset_index(name='count')
    pivot_df = grouped.pivot(index='location_group', columns='crime_group', values='count').fillna(0)
    pivot_df = pivot_df.reindex(columns=crime_groups_to_include, fill_value=0)
    pivot_df['total'] = pivot_df.sum(axis=1)
    pivot_df = pivot_df.sort_values('total', ascending=False).drop(columns='total')

    # Clean x-axis labels to remove "On or near "
    x_labels = [loc.replace("On or near ", "") for loc in pivot_df.index]

    color_map = {
    'Violent / Sexual': '#F7B7A3',
    'Theft / Burglary / Robbery': '#A8D5BA',
    'Anti-Social Behaviour': '#F7E8A3',
    'Drug-Related Crime': '#E2F0CB',
}

    
    # Plot
    fig = go.Figure()
    for crime_group in crime_groups_to_include:
        fig.add_trace(go.Bar(
            name=crime_group,
            x=x_labels,
            y=pivot_df[crime_group],
            marker=dict(color=color_map[crime_group], line=dict(color='black', width=1))
        ))

    fig.update_layout(
        barmode='group',
        title="Crime Counts by Location and Crime Group",
        xaxis_title="Location",
        yaxis_title="Number of Crimes",
        height=700,
        width=1100,
        template='plotly_white',
        legend=dict(
            x=1,         # position on x axis (right)
            y=1,         # position on y axis (top)
            xanchor='right',
            yanchor='top',
            bgcolor='rgba(255,255,255,0.7)',  # slightly transparent background
            bordercolor='black',
            borderwidth=1
        )
    )

    with crime_group_by_location_output:
        crime_group_by_location_output.clear_output()
        display(fig)

def plot_crime_hexbin():
    dff = df.copy()

    # Apply filters
    region = crime_hexbin_filters['region'].value
    month = crime_hexbin_filters['month'].value
    location = crime_hexbin_filters['location'].value
    crime_group = crime_hexbin_filters['crime_group'].value
    outcome_group = crime_hexbin_filters['outcome_group'].value

    if region != 'All':
        dff = dff[dff['region'] == region]
    if month != 'All':
        dff = dff[dff['month_str'] == month]
    if location != 'All':
        dff = dff[dff['lsoa_name_clean'] == location]
    if crime_group != 'All':
        dff = dff[dff['crime_group'] == crime_group]
    if outcome_group != 'All':
        dff = dff[dff['outcome_group'] == outcome_group]

    with crime_hexbin_output:
        crime_hexbin_output.clear_output()

        if dff.empty:
            print("No data matches the selected filters.")
            return

        from matplotlib.colors import LinearSegmentedColormap

        places = {
    'Liverpool': (53.4084, -2.9916),
    'West Kirby': (53.3859, -3.1808),
    'Birkenhead': (53.3933, -3.0136),
    'Wallasey': (53.4161, -3.0606),
    'Heswall': (53.3506, -3.1001),
    'Bromborough': (53.3425, -3.0082),
    'Halewood': (53.3743, -2.8311),
    'Prescot': (53.4394, -2.8234),
    'St Helens': (53.4543, -2.7443),
    'Bootle': (53.4584, -3.0121),
    'Litherland': (53.4590, -3.0208),
    'Crosby': (53.4797, -3.0407),
    'Kirkby': (53.4857, -2.8642),
    'Maghull': (53.5227, -2.9406),
    'Formby': (53.5307, -3.0562),
    'Southport': (53.6453, -3.0081)
}

        
        custom_cmap = LinearSegmentedColormap.from_list(
    'custom_heatmap',
        ['#ffffff', '#ffff00', '#ffa500', '#ff0000']
        )

        
        plt.figure(figsize=(16, 10)) 
        hb = plt.hexbin(
            dff['longitude'], dff['latitude'],
            gridsize=50,
            cmap=custom_cmap,
            norm=mcolors.LogNorm(),
            mincnt=1
        )
        plt.colorbar(label='Number of Crimes (log scale)')
        plt.title(f"Crime Density Hexbin Plot\nRegion: {region}, Month: {month}, Location: {location}\nCrime: {crime_group}, Outcome: {outcome_group}")
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')

        for place, (lat, lon) in places.items():
            plt.plot(lon, lat, 'ko', markersize=5)
            plt.text(
                lon, lat + 0.005, place,
                fontsize=9, fontweight='bold', color='black',
                ha='center', va='bottom', alpha=0.7,
                bbox=dict(facecolor='white', alpha=0.6, edgecolor='none', boxstyle='round,pad=0.3')
            )

        plt.tight_layout()
        plt.show()


# --- Output widgets for displaying figures ---
kpi_output = widgets.Output()
outcome_pie_output = widgets.Output()
crime_pie_output = widgets.Output()
crime_type_output = widgets.Output()
regional_crime_output = widgets.Output()
crime_over_time_output = widgets.Output()
positive_outcomes_by_region_output = widgets.Output()
crime_group_by_location_output = widgets.Output()
crime_hexbin_output = widgets.Output()


# --- Update functions for observers ---

def update_kpi(change=None):
    with kpi_output:
        kpi_output.clear_output(wait=True)
        fig = plot_kpi()
        fig.show()

def update_outcome_pie(change=None):
    with outcome_pie_output:
        outcome_pie_output.clear_output(wait=True)
        fig = plot_outcome_pie()
        if fig:
            fig.show()
        else:
            print("No data for selected filters")

def update_crime_pie(change=None):
    with crime_pie_output:
        crime_pie_output.clear_output(wait=True)
        fig = plot_crime_group_pie()
        if fig:
            fig.show()
        else:
            print("No data for selected filters")

def update_crime_type(change=None):
    plot_crime_type()

def update_regional_crime(change=None):
    plot_regional_crime()

def update_crime_over_time(change=None):
    plot_crime_over_time()

def update_positive_outcomes_by_region(change=None):
    plot_positive_outcomes_by_region()

def update_crime_group_by_location(change=None):
    plot_crime_group_by_location()   

def update_crime_hexbin(change=None):
    plot_crime_hexbin() 

# --- Attach observers for each filter group ---

for widget_ in kpi_filters.values():
    widget_.observe(update_kpi, names='value')

for widget_ in outcome_pie_filters.values():
    widget_.observe(update_outcome_pie, names='value')

for widget_ in crime_pie_filters.values():
    widget_.observe(update_crime_pie, names='value')

for widget_ in crime_type_filters.values():
    widget_.observe(update_crime_type, names='value')

for widget_ in regional_crime_filters.values():
    widget_.observe(update_regional_crime, names='value')

for widget_ in crime_over_time_filters.values():
    widget_.observe(update_crime_over_time, names='value')

for widget_ in positive_outcomes_by_region_filters.values():
    widget_.observe(update_positive_outcomes_by_region, names='value')

for widget_ in crime_group_by_location_filters.values():
    widget_.observe(update_crime_group_by_location, names='value')

for widget_ in crime_hexbin_filters.values():
    widget_.observe(update_crime_hexbin, names='value')

# --- Display all widgets and plots ---

dashboard = widgets.VBox([
    
    widgets.Label("Total Crimes KPI Filters"),
    widgets.HBox(list(kpi_filters.values())),
    kpi_output,
    
    widgets.Label("Outcome Group Pie Filters"),
    widgets.HBox(list(outcome_pie_filters.values())),
    outcome_pie_output,
    
    widgets.Label("Crime Group Pie Filters"),
    widgets.HBox(list(crime_pie_filters.values())),
    crime_pie_output,

    widgets.Label("Crime Type Distribution Filters"),
    widgets.HBox(list(crime_type_filters.values())),
    crime_type_output,

    widgets.Label("Regional Crime Volume Filters"),
    widgets.HBox(list(regional_crime_filters.values())),
    regional_crime_output,

    widgets.Label("Crime Over Time Filters"),
    widgets.HBox(list(crime_over_time_filters.values())),
    crime_over_time_output,

    widgets.Label("Positive Outcome Rates Filters"),
    widgets.HBox(list(positive_outcomes_by_region_filters.values())),
    positive_outcomes_by_region_output,

    widgets.Label("Crime Group by Location Filters"),
    widgets.HBox(list(crime_group_by_location_filters.values())),
    crime_group_by_location_output,

    widgets.Label("Merseyside Crime Hexbin Filters"),
    widgets.HBox(list(crime_hexbin_filters.values())),
    crime_hexbin_output,
])

display(dashboard)

# Initial plot draws
update_kpi()
update_outcome_pie()
update_crime_pie()
update_crime_type()
update_regional_crime()
update_crime_over_time()
update_positive_outcomes_by_region()
update_crime_group_by_location()
update_crime_hexbin()